In [2]:
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
# import keras_tuner

In [60]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

data = data.drop(
    columns=["release_date", "key", "loudness",
             "explicit", "popularity", "duration_ms", "danceability", "energy", "speechiness",
             "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

data.head(5)

,user_id,track_id,session_id,favourite_genres,genres,skipped
0,101,2PmGtDUyJIpYBEtI1hQIVp,124,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, folk rock...",False
1,103,2PmGtDUyJIpYBEtI1hQIVp,151,"[filmi, regional mexican, folk]","[album rock, art rock, classic rock, folk rock...",False
2,132,2PmGtDUyJIpYBEtI1hQIVp,544,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False
3,132,5yxYokipsWlpDCt4Th4VVc,534,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False
4,132,5yxYokipsWlpDCt4Th4VVc,547,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False


In [61]:
# unique_favourite_genres = set()
# for genres in data['favourite_genres']:
#   for genre in genres:
#     unique_favourite_genres.add(genre)
#
# unique_genres = set()
# for genres in data['genres']:
#   for genre in genres:
#     unique_genres.add(genre)
#
# # show all common genres between all favourite_genres and genres
# print("unique_favourite_genres", len(unique_favourite_genres))
# print("unique_genres", len(unique_genres))
# print("common genres", unique_favourite_genres.intersection(unique_genres))
#
# # remove genres that are not in favourite_genres
# data['genres'] = data['genres'].apply(
#     lambda x: [genre for genre in x if genre in unique_favourite_genres])

# TODO remove ??? moze niepotrzebne

unique_favourite_genres 46
unique_genres 1766
common genres {'j-pop', 'metal', 'latin', 'regional mexican', 'mandopop', 'brill building pop', 'latin alternative', 'hard rock', 'permanent wave', 'pop', 'new wave pop', 'tropical', 'rock en espanol', 'singer-songwriter', 'turkish pop', 'filmi', 'soul', 'classic rock', 'alternative metal', 'new romantic', 'art rock', 'lounge', 'soft rock', 'vocal jazz', 'europop', 'psychedelic rock', 'rock', 'pop rock', 'mpb', 'ranchera', 'alternative rock', 'c-pop', 'quiet storm', 'hoerspiel', 'new wave', 'latin pop', 'country rock', 'album rock', 'blues rock', 'folk', 'motown', 'latin rock', 'funk', 'mellow gold', 'adult standards', 'dance pop'}


In [1]:
# Combine genres and favourite_genres
all_genres = list(data['favourite_genres'] + data['genres'])

# One-hot encode the genres
mlb = MultiLabelBinarizer()
mlb.fit(all_genres)

# encoded_all_genres = mlb.fit_transform(all_genres)

# Split encoded_genres into genres and favourite_genres
# encoded_favourite_genres = encoded_all_genres[:, :len(data['favourite_genres'][0])]
# encoded_genres = encoded_all_genres[:, len(data['favourite_genres'][0]):]

encoded_favourite_genres = mlb.transform(data['favourite_genres'][0])
encoded_genres = mlb.transform(data['genres'][0])

# create data frame from data genres, data favourite_genres, encoded genres, encoded favourite_genres
df = pd.DataFrame(data={'genres': data['genres'], 'favourite_genres': data['favourite_genres'],
                        'encoded_genres': encoded_genres,
                        'encoded_favourite_genres': encoded_favourite_genres})

df.head(5)

NameError: name 'data' is not defined

In [63]:
# TODO uzywac keras tokenizer?

# Concatenate the one-hot encoded columns
X = np.concatenate([encoded_favourite_genres, encoded_genres], axis=1)

# Extract the labels
y = data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# count skipped and not skipped songs in X_train
print("X_train skipped", np.count_nonzero(y_train == 1))
print("X_train not skipped", np.count_nonzero(y_train == 0))
print("X_train not skipped %", np.count_nonzero(y_train == 0) / len(y_train))


X_train skipped 2905
X_train not skipped 4999
X_train not skipped % 0.6324645748987854


In [64]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(46 * 2, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(46, activation='relu'))
  model.add(Dropout(0.5))
  # model.add(Dense(1000, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # model.compile(
  #     optimizer=keras.optimizers.Adam(hp.Choice('learning_date', values=[0.5, 0.1, 0.01])),
  #     loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(
      optimizer=Adam(),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [66]:
epochs = 50
batch_size = 32

model = build_model(None)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

# tuner = keras_tuner.tuners.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=50,
#     max_trials=10,
#     executions_per_trial=2,
#     directory='my_dir')


Epoch 1/50
198/198 - 3s - loss: 0.6480 - accuracy: 0.6250 - val_loss: 0.6256 - val_accuracy: 0.6762 - 3s/epoch - 13ms/step
Epoch 2/50
198/198 - 1s - loss: 0.6121 - accuracy: 0.6714 - val_loss: 0.6031 - val_accuracy: 0.6818 - 1s/epoch - 6ms/step
Epoch 3/50
198/198 - 1s - loss: 0.5900 - accuracy: 0.6895 - val_loss: 0.5934 - val_accuracy: 0.6837 - 1s/epoch - 6ms/step
Epoch 4/50
198/198 - 1s - loss: 0.5819 - accuracy: 0.6853 - val_loss: 0.5891 - val_accuracy: 0.6932 - 1s/epoch - 6ms/step
Epoch 5/50
198/198 - 1s - loss: 0.5773 - accuracy: 0.6963 - val_loss: 0.5829 - val_accuracy: 0.6901 - 1s/epoch - 6ms/step
Epoch 6/50
198/198 - 1s - loss: 0.5760 - accuracy: 0.6935 - val_loss: 0.5829 - val_accuracy: 0.6856 - 1s/epoch - 6ms/step
Epoch 7/50
198/198 - 1s - loss: 0.5739 - accuracy: 0.7003 - val_loss: 0.5801 - val_accuracy: 0.6799 - 1s/epoch - 6ms/step
Epoch 8/50
198/198 - 1s - loss: 0.5667 - accuracy: 0.7041 - val_loss: 0.5790 - val_accuracy: 0.6970 - 1s/epoch - 6ms/step
Epoch 9/50
198/198 - 1s

In [ ]:
# tuner.search(
#     (X_train, y_train),
#     validation_data=(X_test, y_test),
# )

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test set accuracy:", accuracy)

print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)
#
# # Print the predicted and actual labels
# print("Predicted labels:", y_pred_classes.flatten())
# print("Actual labels:", y_test)

In [ ]:
# # test with new data
#
# new_data = [
#   (["dominican pop", "merengue", "merengue tipico", "tropical"],
#    ["blues rock", "country rock", "lounge"])
# ]
#
# labels = [
#
# ]
#
# new_df = pd.DataFrame(new_data, columns=["genres", "favourite_genres"])
#
# # Combine genres and favourite_genres
# all_new_genres = list(new_df['genres'] + new_df['favourite_genres'])
#
# # One-hot encode the genres using the previously fit MultiLabelBinarizer (mlb)
# encoded_new_genres = mlb.transform(all_new_genres)
#
# # Split encoded_new_genres into genres and favourite_genres
# encoded_new_genres1 = encoded_new_genres[:, :len(new_df['genres'][0])]
# encoded_new_genres2 = encoded_new_genres[:, len(new_df['genres'][0]):]
#
# # Concatenate the one-hot encoded columns
# X_new = np.concatenate([encoded_new_genres1, encoded_new_genres2], axis=1)
#
# y_new_pred = model.predict(X_new)
# y_new_pred_classes = (y_new_pred > 0.5).astype(int)
#
# # Print the predicted labels
# print("Predicted labels:", y_new_pred_classes.flatten())
# print("Actual labels:", labels)
